In [168]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle
import torch
from tqdm import tqdm

In [205]:
df=pd.read_csv("Churn_Modelling.csv")

In [206]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [207]:
df_1=df[df["Exited"]==1]
df_2=df[df["Exited"]==0]
df_2=df_2.iloc[:2050,:]

In [208]:
df=pd.concat([df_1,df_2],axis=0)

In [143]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
16,17,15737452,Romeo,653,Germany,Male,58,1,132602.88,1,1,0,5097.67,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2599,2600,15704844,Hsiung,550,Spain,Male,62,7,80927.56,1,0,1,64490.67,0
2601,2602,15616700,Leach,622,Spain,Female,41,9,0.00,2,1,1,155786.39,0
2602,2603,15683521,Godfrey,594,Germany,Male,28,0,142574.71,2,1,0,129084.82,0
2603,2604,15583049,Wallace,643,Germany,Female,34,7,160426.07,1,0,1,188533.11,0


In [209]:
df=df.drop(["RowNumber","CustomerId","Surname"],axis=1)

In [210]:
df=df.reset_index(drop=True)

In [211]:
label_encoder=LabelEncoder()
df["Gender"]=label_encoder.fit_transform(df["Gender"])

In [212]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,502,France,0,42,8,159660.80,3,1,0,113931.57,1
2,645,Spain,1,44,8,113755.78,2,1,0,149756.71,1
3,376,Germany,0,29,4,115046.74,4,1,0,119346.88,1
4,653,Germany,1,58,1,132602.88,1,1,0,5097.67,1
...,...,...,...,...,...,...,...,...,...,...,...
4082,550,Spain,1,62,7,80927.56,1,0,1,64490.67,0
4083,622,Spain,0,41,9,0.00,2,1,1,155786.39,0
4084,594,Germany,1,28,0,142574.71,2,1,0,129084.82,0
4085,643,Germany,0,34,7,160426.07,1,0,1,188533.11,0


In [213]:
from sklearn.preprocessing import OneHotEncoder

In [214]:
ohe=OneHotEncoder(sparse_output=False)

In [215]:
ohencoded=ohe.fit_transform(df[["Geography"]])

In [216]:
ohe.get_feature_names_out()

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [217]:
ohencoded.shape

(4087, 3)

In [218]:
new_df=pd.DataFrame(ohencoded,columns=ohe.get_feature_names_out())

In [219]:
new_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,0.0,0.0,1.0
3,0.0,1.0,0.0
4,0.0,1.0,0.0
...,...,...,...
4082,0.0,0.0,1.0
4083,0.0,0.0,1.0
4084,0.0,1.0,0.0
4085,0.0,1.0,0.0


In [220]:
concat_df=pd.concat([df,new_df],axis=1)

In [221]:
concat_df=concat_df.drop(["Geography"],axis=1)

In [223]:
with open("one_hot_encoder.pkl","wb") as f:
    pickle.dump(ohe,f)
with open("label_encoder.pkl","wb") as f:
    pickle.dump(label_encoder,f)

In [224]:
X=concat_df.drop(["Exited"],axis=1)
Y=concat_df["Exited"]
SS=StandardScaler()
X=SS.fit_transform(X)
X=torch.Tensor(X)
Y=torch.Tensor(Y)
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)

In [225]:
with open("Standard_Scalar.pkl","wb") as f:
    pickle.dump(SS,f)

In [226]:
from torch.utils.data import DataLoader,TensorDataset
training_data=TensorDataset(X_train,Y_train)
testing_data=TensorDataset(X_test,Y_test)
training_dataloader=DataLoader(training_data,batch_size=8,shuffle=True)
testing_dataloader=DataLoader(testing_data,batch_size=8)

In [227]:
next(iter(training_dataloader))

[tensor([[ 0.6538, -1.0037,  1.0388, -1.0414, -1.3498, -0.7615, -1.5429,  1.0787,
           0.7500,  1.1018, -0.6718, -0.5579],
         [-0.1285,  0.9963, -0.1018,  1.3700,  1.1676, -0.7615,  0.6481,  1.0787,
           0.4249, -0.9076,  1.4885, -0.5579],
         [-2.0690, -1.0037,  0.3735,  1.7145,  0.6926, -0.7615,  0.6481,  1.0787,
           0.4758, -0.9076, -0.6718,  1.7926],
         [ 0.7350,  0.9963, -0.6722,  1.0255, -1.3498,  0.7276,  0.6481,  1.0787,
           0.5659,  1.1018, -0.6718, -0.5579],
         [ 0.1255,  0.9963,  1.7042, -1.0414,  0.3561, -0.7615,  0.6481, -0.9270,
          -1.6129, -0.9076,  1.4885, -0.5579],
         [-0.8498,  0.9963, -0.4820, -1.3859,  0.9688, -0.7615,  0.6481, -0.9270,
          -0.3695,  1.1018, -0.6718, -0.5579],
         [-0.2911, -1.0037,  0.1833, -0.0079, -1.3498, -0.7615,  0.6481,  1.0787,
          -0.9262,  1.1018, -0.6718, -0.5579],
         [-0.7482, -1.0037, -0.0068, -0.3524,  0.4873, -0.7615, -1.5429, -0.9270,
           1.56

In [271]:
from torch import nn
from torch.utils.tensorboard import SummaryWriter
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.Layers=nn.Sequential(
            nn.Linear(in_features=12,out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128,out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64,out_features=32),
            nn.ReLU(),
            nn.Linear(in_features=32,out_features=1),
            nn.ReLU()
        )
    def forward(self,x):
        return self.Layers(x)

In [272]:
torch.manual_seed(42)
model=Model()

In [273]:
loss_fn=nn.BCEWithLogitsLoss()
optimizer=torch.optim.SGD(params=model.parameters(),lr=0.01)

In [274]:
from torchmetrics import Accuracy
torchmetrics_accuracy=Accuracy(task="binary")

In [275]:
overall_training_acc=[]
overall_training_loss=[]
overall_testing_acc=[]
overall_testing_loss=[]

In [276]:
epochs=50
for epoch in tqdm(range(epochs)):
    model.train()
    training_acc_per_epoch=[]
    training_loss_per_epoch=[]
    testing_acc_per_epoch=[]
    testing_loss_per_epoch=[]
    for X,Y in training_dataloader:
        logits=model(X)
        loss=loss_fn(logits.squeeze(),Y)
        prediction=torch.round(torch.sigmoid(logits))
        accuracy=torchmetrics_accuracy(prediction.squeeze(),Y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        training_acc_per_epoch.append(accuracy)
        training_loss_per_epoch.append(loss.item())
    model.eval()
    with torch.inference_mode():
        for X,Y in testing_dataloader:
            logits=model(X)
            loss=loss_fn(logits.squeeze(),Y)
            prediction=torch.round(torch.sigmoid(logits))
            accuracy=torchmetrics_accuracy(prediction.squeeze(),Y)
            testing_acc_per_epoch.append(accuracy)
            testing_loss_per_epoch.append(loss.item())
    training_accuracy=sum(training_acc_per_epoch)/len(training_acc_per_epoch)
    training_loss=sum(training_loss_per_epoch)/len(training_loss_per_epoch)
    testing_accuracy=sum(testing_acc_per_epoch)/len(testing_acc_per_epoch)
    testing_loss=sum(testing_loss_per_epoch)/len(testing_loss_per_epoch)
    print(f"Training_Accuracy:{training_accuracy}|Training_loss:{training_loss}|Testing_loss:{testing_loss}|Testing_accuracy:{testing_accuracy}")

  2%|▏         | 1/50 [00:00<00:37,  1.31it/s]

Training_Accuracy:0.5112469792366028|Training_loss:0.6893770627695657|Testing_loss:0.6873189749069584|Testing_accuracy:0.49878641963005066


  4%|▍         | 2/50 [00:01<00:37,  1.30it/s]

Training_Accuracy:0.5504890084266663|Training_loss:0.6801666403166531|Testing_loss:0.6758848920609187|Testing_accuracy:0.5497573018074036


  6%|▌         | 3/50 [00:02<00:36,  1.30it/s]

...

Training_Accuracy:0.805623471736908|Training_loss:0.5204056275867308|Testing_loss:0.5771637475027621|Testing_accuracy:0.7609223127365112


 98%|█████████▊| 49/50 [00:37<00:00,  1.30it/s]

Training_Accuracy:0.8053178191184998|Training_loss:0.518211179556357|Testing_loss:0.5914817559487612|Testing_accuracy:0.7439320683479309


100%|██████████| 50/50 [00:38<00:00,  1.30it/s]

Training_Accuracy:0.8066014647483826|Training_loss:0.516837948691874|Testing_loss:0.5647759943911173|Testing_accuracy:0.7633495330810547


In [277]:
torch.save(model.state_dict(),"Model_state.pth")

In [360]:
df["Exited"].value_counts()

Exited
0    7963
1    2037
Name: count, dtype: int64